In [1]:
%env CUDA_VISIBLE_DEVICES=1,2,3
!pip install datasets transformers[sentencepiece]

env: CUDA_VISIBLE_DEVICES=1,2,3
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/centos/python/3.6.2/bin/python3.6 -m pip install --upgrade pip' command.


In [2]:
from datasets import load_dataset, ClassLabel, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import torch
import os


In [3]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')
metric = load_metric("accuracy")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [4]:
# xzl: this???
os.environ["WANDB_DISABLED"] = "true"

def to_bin_class(ex):
	ex['label'] = round(ex['label'])
	return ex

def tokenize_fn(ex):
	return tokenizer(ex['sentence'], padding='max_length', truncation=True)

def compute_metrics(eval_pred):
	logits, labels = eval_pred
	preds = np.argmax(logits, axis=-1)
	return metric.compute(predictions=preds, references=labels)

xzl: prep data....

In [5]:
sst = load_dataset('sst', 'default')
sst = sst.remove_columns(['tokens', 'tree'])
sst = sst.map(to_bin_class)
sst = sst.cast_column('label', ClassLabel(num_classes=2))


Reusing dataset sst (/u/xl6yq/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /u/xl6yq/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-175e01bd44ae6e77.arrow
Loading cached processed dataset at /u/xl6yq/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-bcfe3eb7a84204f3.arrow
Loading cached processed dataset at /u/xl6yq/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-d17996794240dda4.arrow
Loading cached processed dataset at /u/xl6yq/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-a4488ee52a49920a.arrow
Loading cached processed dataset at /u/xl6yq/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-1c1de7b235a26b22.arrow
Loading cached processed dataset at /u/xl6yq/.cache/huggingface/datasets/ss

In [6]:
sst_tokenized = sst.map(tokenize_fn, batched=True)

# xzl: remove more
sst_tokenized = sst_tokenized.remove_columns(['sentence'])

trn_set = sst_tokenized['train']
tst_set = sst_tokenized['test']
val_set = sst_tokenized['validation']

Loading cached processed dataset at /u/xl6yq/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-4fde5a6aea4f6c63.arrow
Loading cached processed dataset at /u/xl6yq/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-8907ad2a7135934b.arrow
Loading cached processed dataset at /u/xl6yq/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-8432e9f63b9a4002.arrow


xzl: check the columns...

In [7]:
print(val_set.column_names)

['label', 'input_ids', 'attention_mask']


In [8]:
# batch ...
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    trn_set, shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    val_set, batch_size=8, collate_fn=data_collator
)

In [10]:
# check data ...
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}    

{'input_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512]),
 'labels': torch.Size([8])}

run one batch

In [11]:
# run one batch
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)
print(outputs)

tensor(0.6943, grad_fn=<NllLossBackward0>) torch.Size([8, 2])
SequenceClassifierOutput(loss=tensor(0.6943, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0187,  0.0525],
        [ 0.0408,  0.0420],
        [ 0.0791,  0.0350],
        [ 0.0202,  0.0627],
        [ 0.0858,  0.0778],
        [ 0.0481,  0.0707],
        [ 0.0414,  0.0932],
        [ 0.0063,  0.0698]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [12]:
#from transformers import AdamW    # deprecated
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [13]:
from transformers import get_scheduler

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1068


In [14]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [15]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1068 [00:00<?, ?it/s]

validation....

In [16]:
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.851952770208901}